In [34]:
from hashlib import sha256
import json
import time

In [68]:
class BlockException(Exception):
    pass

In [35]:
class Block():
    
    def __init__(self, index, timestamp, transactions, proof, previous_block):
        self.index = index
        self.timestamp = timestamp
        self.transactions = transactions
        self.proof = proof
        self.previous_block = previous_block
        self.hash = self.get_hash()
        
    def get_hash(self):
        if self.transactions:
            block_string = "{}{}{}".format(self.index,
                                           self.timestamp,
                                           json.dumps(self.transactions))
        else:
            block_string = "{}{}".format(self.index,
                                         self.timestamp)
            
        hash_value = sha256(block_string.encode()).hexdigest()
        return hash_value
        
    

In [81]:
class Blockchain():
    
    def __init__(self):
        self.transactions = []
        self.last_block = Blockchain.create_genesis_block()
        
    def add_block(self):
        if not self.transactions:
            raise BlockException("No transactions to add to block")
        new_block = Blockchain.mine_block(self.transactions, self.last_block)
        self.last_block = new_block
        
    def add_transaction(self, sender, recipient, amount, timestamp=None):
        if not timestamp:
            timestamp = time.time()
        transaction = {"sender": sender,
                      "recipient": recipient,
                      "amount": amount,
                      "timestamp": timestamp}
        self.transactions.append(transaction)
        
    @staticmethod
    def find_proof(hash_string, starting_value=0):
        proof = starting_value
        proof_found = False
        while not proof_found:
            hash_value = sha256("{}{}".format(hash_string, proof).encode()).hexdigest()
            if hash_value[:4] == "0000":
                proof_found = True
            proof += 1
        
        return proof
    
    @staticmethod
    def create_genesis_block():
        index = 0
        timestamp = time.time()
    
        genesis_hash = sha256("{}{}".format(index, timestamp).encode()).hexdigest()
    
        proof = 0
        proof_found = False
        while not proof_found:
            hash_value = sha256("{}{}".format(genesis_hash, proof).encode()).hexdigest()
            if hash_value[:4] == "0000":
                proof_found = True
            proof += 1
    
        genesis_block = Block(0, timestamp, None, proof, None)
        return genesis_block
    
    @staticmethod
    def mine_block(data, previous_block):
        index = previous_block.index + 1
        timestamp = time.time()
        block_string = "{}{}{}".format(index,
                                      timestamp,
                                      json.dumps(data))
        proof = Blockchain.find_proof(block_string)
        new_block = Block(index, timestamp, data, proof, previous_block)
        return new_block

In [82]:
dumbcoin = Blockchain()
dumbcoin.add_transaction("tyler", "luke", 50)
dumbcoin.add_transaction("luke", "tyler", 10.10)

dumbcoin.add_block()

dumbcoin.add_transaction("tyler", "olivia", 0.10)
dumbcoin.add_transaction("olivia", "narendra", 11.10)

dumbcoin.add_block()

dumbcoin.add_transaction("cecily", "narendra", 0.10)
dumbcoin.add_transaction("olivia", "narendra", 11.10)

dumbcoin.add_block()

dumbcoin.add_transaction("olivia", "narendra", 0.10)
dumbcoin.add_transaction("luke", "tyler", 11.10)

dumbcoin.add_block()

In [86]:
def print_blockchain(block):
    if block.transactions:
        print(block.transactions)
    if block.previous_block:
        print_blockchain(block.previous_block)

In [87]:
print_blockchain(dumbcoin.last_block)

[{'sender': 'tyler', 'recipient': 'luke', 'amount': 50, 'timestamp': 1516252197.061805}, {'sender': 'luke', 'recipient': 'tyler', 'amount': 10.1, 'timestamp': 1516252197.061834}, {'sender': 'tyler', 'recipient': 'olivia', 'amount': 0.1, 'timestamp': 1516252232.610523}, {'sender': 'olivia', 'recipient': 'narendra', 'amount': 11.1, 'timestamp': 1516252232.610564}]
[{'sender': 'tyler', 'recipient': 'luke', 'amount': 50, 'timestamp': 1516252197.061805}, {'sender': 'luke', 'recipient': 'tyler', 'amount': 10.1, 'timestamp': 1516252197.061834}, {'sender': 'tyler', 'recipient': 'olivia', 'amount': 0.1, 'timestamp': 1516252232.610523}, {'sender': 'olivia', 'recipient': 'narendra', 'amount': 11.1, 'timestamp': 1516252232.610564}]
